In [1]:
%cd ~/protein-transfer

/home/francesca/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from scr.utils import pickle_load

In [8]:
result_dcit = pickle_load("results/sklearn-carp/proeng/gb1/sampled/carp_600k/mean/carp_600k-mean-layer_15.pkl")
result_dcit['train']['mse']

1.9969547196153044

In [10]:
result_dcit = pickle_load("results/sklearn-carp-0.125/proeng/gb1/sampled/carp_600k/mean/carp_600k-mean-layer_15.pkl")
result_dcit['train']['mse']

1.9636935465904168

In [11]:
from scr.analysis.perlayer import LayerLoss

In [12]:
carp_layerloss = LayerLoss(
    input_path="results/sklearn-carp",
    output_path="results/sklearn-carp_layer"
)

In [19]:
carp_layerloss.layer_analysis_dict

defaultdict(dict,
            {'proeng_aav_two_vs_many_mean': {'carp_76M': {'train_mse': array([19.98643466, 16.91308442, 16.61697172, 16.49396424, 16.45759354,
                      16.46137655, 16.47681066, 16.52387118, 16.55400994, 16.54224003,
                      16.52699632, 16.51819479, 16.50368693, 16.50645948, 16.51018905,
                      16.51834378, 16.53849288, 16.54305334, 16.53029264, 16.5312803 ,
                      16.52736704, 16.52958116, 16.53141356, 16.53131422, 16.54122177,
                      16.54382726, 16.54563925, 16.55259983, 16.56480337, 16.58029155,
                      16.5897708 , 16.61731492, 16.62080324]),
               'val_mse': array([20.27781688, 17.22433767, 16.87588968, 16.69268243, 16.69217913,
                      16.6975683 , 16.73511578, 16.78779179, 16.80398083, 16.79106107,
                      16.81015078, 16.80013428, 16.77013057, 16.77629521, 16.77682976,
                      16.77165833, 16.76987814, 16.779343  , 16.76451

In [4]:
LayerLoss(
    input_path="results/pytorch-esm",
    output_path="results/pytorch-esm_layer"
)

In [4]:
LayerLoss(
    input_path="results/pytorch-carp",
    output_path="results/pytorch-carp_layer"
)

Making results/pytorch-carp_layer/annotation/scl/balanced/carp_640M ...
Making results/pytorch-carp_layer/annotation/scl/balanced/carp_640M/mean ...
Making results/pytorch-carp_layer-rand/annotation/scl/balanced/carp_640M ...
Making results/pytorch-carp_layer-rand/annotation/scl/balanced/carp_640M/mean ...
Making results/pytorch-carp_layer-stat/annotation/scl/balanced/carp_640M ...
Making results/pytorch-carp_layer-stat/annotation/scl/balanced/carp_640M/mean ...


In [6]:
LayerLoss(
    input_path="test/sklearn-scalex-scaley-noloader-fixembpool",
    output_path="test/sklearn-scalex-scaley-noloader-fixembpool_layer")

In [ ]:
LayerLoss(
    input_path="test/sklearn",
    output_path="test/sklearn_layer")

In [5]:
LayerLoss(
    input_path="test/sklearn-scaley-noloader-fixembpool",
    output_path="test/sklearn-scaley-noloader-fixembpool_layer")

Making test/sklearn-scaley-noloader-fixembpool_layer-onehot ...
Making test/sklearn-scaley-noloader-fixembpool_layer-onehot/proeng ...
Making test/sklearn-scaley-noloader-fixembpool_layer-onehot/proeng/thermo ...
Making test/sklearn-scaley-noloader-fixembpool_layer-onehot/proeng/thermo/mixed_split ...
Making test/sklearn-scaley-noloader-fixembpool_layer-onehot/proeng/thermo/mixed_split/onehot ...
Making test/sklearn-scaley-noloader-fixembpool_layer-onehot/proeng/thermo/mixed_split/onehot/flatten ...


In [13]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scalex-scaley-noloader-fixembpool",
    output_path="results/sklearn-scalex-scaley-noloader-fixembpool_layer",
).layer_analysis_dict

In [21]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader-fixembpool",
    output_path="results/sklearn-scaley-noloader-fixembpool_layer",
).layer_analysis_dict

In [7]:
layer_analysis_dict.keys()

dict_keys(['proeng_gb1_two_vs_rest_mean', 'proeng_gb1_low_vs_high_mean', 'proeng_gb1_sampled_mean', 'proeng_aav_one_vs_many_mean', 'proeng_aav_two_vs_many_mean'])

In [8]:
layer_analysis_dict["proeng_gb1_sampled_mean"].keys()

dict_keys(['esm1_t34_670M_UR50S', 'esm1_t12_85M_UR50S', 'esm1b_t33_650M_UR50S', 'esm1_t6_43M_UR50S'])

In [9]:
layer_analysis_dict["proeng_gb1_sampled_mean"]["esm1_t6_43M_UR50S"].keys()

dict_keys(['train_mse', 'val_mse', 'test_mse', 'test_ndcg', 'test_rho'])

In [10]:
layer_analysis_dict["proeng_gb1_sampled_mean"]["esm1_t6_43M_UR50S"]["test_rho"]

array([       nan, 0.78954326, 0.82729954, 0.84882642, 0.85467853,
       0.84874573, 0.84821129])

In [11]:
from scr.utils import pickle_load

In [15]:
gb1_sample_output_dict = pickle_load("results/sklearn-scaley-noloader-fixembpool/proeng/gb1/sampled/esm1_t6_43M_UR50S/mean/esm1_t6_43M_UR50S-mean-layer_0.pkl")
gb1_sample_output_dict.keys(), gb1_sample_output_dict["test"].keys()

(dict_keys(['train', 'val', 'test']),
 dict_keys(['mse', 'pred', 'true', 'ndcg', 'rho']))

In [16]:
from scipy.stats import spearmanr

In [18]:
gb1_sample_output_dict["test"]["true"]

array([1.69016355e+00, 5.85648854e-01, 3.05454865e+00, ...,
       2.25273974e-03, 2.62819636e-02, 4.49316942e-03])

In [19]:
gb1_sample_output_dict["test"]["pred"]

array([4.51927324e-17, 4.51927324e-17, 4.51927324e-17, ...,
       4.51927324e-17, 4.51927324e-17, 4.51927324e-17])

In [17]:
spearmanr(gb1_sample_output_dict["test"]["true"], gb1_sample_output_dict["test"]["pred"])

/anaconda/envs/protran/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4529: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


SpearmanrResult(correlation=nan, pvalue=nan)

In [6]:
layer_analysis_dict = LayerLoss(
    input_path="results/train_val_test",
    output_path="results/analysis_layer",
).layer_analysis_dict

In [5]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scalex-scaley-noloader",
    output_path="results/sklearn-scalex-scaley-noloader_layer",
).layer_analysis_dict

In [4]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader",
    output_path="results/sklearn-scaley-noloader_layer",
).layer_analysis_dict

In [10]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader-rand",
    output_path="results/sklearn-scaley-noloader-rand_layer",
).layer_analysis_dict

Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1b_t33_650M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1b_t33_650M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t6_43M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t6_43M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t12_85M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t12_85M_UR50S/mean ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t34_670M_UR50S ...
Making results/sklearn-scaley-noloader-rand_layer/proeng/gb1/sampled/esm1_t34_670M_UR50S/mean ...


In [6]:
from scr.params.emb import TRANSFORMER_INFO

In [9]:
[""] + list(TRANSFORMER_INFO.keys())

['',
 'esm1_t6_43M_UR50S',
 'esm1_t12_85M_UR50S',
 'esm1_t34_670M_UR50S',
 'esm1b_t33_650M_UR50S']

In [8]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-fixa-noloader",
    output_path="results/sklearn-scaley-fixa-noloader_layer",
).layer_analysis_dict

In [6]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-fixa",
    output_path="results/sklearn-scaley-fixa_layer",
).layer_analysis_dict

In [32]:
layer_analysis_dict.keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"].keys(), layer_analysis_dict["proeng_gb1_low_vs_high_mean"]["esm1b_t33_650M_UR50S"].keys()

(dict_keys(['proeng_gb1_two_vs_rest_mean', 'proeng_gb1_low_vs_high_mean', 'proeng_aav_one_vs_many_mean']),
 dict_keys(['esm1b_t33_650M_UR50S', 'esm1_t6_43M_UR50S', 'esm1_t12_85M_UR50S', 'esm1_t34_670M_UR50S']),
 dict_keys(['train_mse', 'test_ndcg', 'test_rho']))

In [26]:
layer_analysis_dict = LayerLoss(
    input_path="results/sklearn-scaley-noloader",
    output_path="results/sklearn-scaley-noloader_layer",
).layer_analysis_dict

Making results/sklearn-scaley-noloader_layer-onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1 ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/sampled/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/two_vs_rest/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high/onehot ...
Making results/sklearn-scaley-noloader_layer-onehot/proeng/gb1/low_vs_high/onehot/flatten ...
Making results/sklearn-scaley-noloader_layer-oneh